In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        print(event.dict())
        print("")
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x1577b2f50>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

QueryStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 776495), 'id_': UUID('278cdd77-1ddb-4645-83ce-aae1bbfe04bc')}

RetrievalStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 778082), 'id_': UUID('af7e88ef-59bf-4ee3-8d62-551078752ecc')}

RetrievalEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 963703), 'id_': UUID('81135615-eb82-4bf0-9f9e-c2756002189a')}

SynthesizeStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 964581), 'id_': UUID('4fa8e49f-88d0-49fc-9cda-89d586ba952b')}

GetResponseStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 975952), 'id_': UUID('2283feee-3ce3-42e1-b789-8a5a315e2652')}

LLMPredictStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 5, 980634), 'id_': UUID('592d347e-d148-467f-837b-7ff582ec5a2f')}

LLMPredictEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 7, 576994), 'id_': UUID('132b8211-bb60-4c80-bb8e-068593083b17')}

GetResponseEndEvent
{'timestam

In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-29db3ffd-8b6d-4fbb-a3b4-8e8083beecc4', parent_id='BaseRetriever.retrieve-a6932e9b-923e-4bdb-bd9b-8d7851487807', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 778394), end_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 963457), duration=0.185063),
 SimpleSpan(id_='BaseRetriever.retrieve-a6932e9b-923e-4bdb-bd9b-8d7851487807', parent_id='RetrieverQueryEngine._query-f115e05c-df1f-4e83-8848-bf0bc5e51f46', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 778061), end_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 964456), duration=0.186395),
 SimpleSpan(id_='LLM.predict-69c3336f-fd4b-4ab2-ac2a-66051620cc7d', parent_id='Refine.get_response-b62868e4-ee86-49b4-85b3-d6a542c9d83c', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 980594), end_time=datetime.datetime(2024, 3, 11, 17, 49, 7, 577694), duration=1.5971),
 SimpleSpan(id_='Refine.get_response-b62868e4-ee86-49b4-85b3-d6a542c9d83c', parent_id='CompactAndRefine.get_response-b34a

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 74694), 'id_': UUID('14dc639f-7198-43bf-956e-71f27d55a9ac')}

RetrievalStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 76843), 'id_': UUID('3780a57d-38b1-454f-a66d-55b27aff2906')}

RetrievalEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 270904), 'id_': UUID('4df1e26c-badf-416a-83e5-8e4d2521e13c')}

SynthesizeStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 271904), 'id_': UUID('d14b7aa7-47d1-4f49-b7ad-4f37352b10c2')}

GetResponseStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 279373), 'id_': UUID('ec509571-6d85-4c83-9671-43317a478981')}

LLMPredictStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 9, 283686), 'id_': UUID('bccb9803-ed68-480f-ad05-02bbca08d7e6')}

LLMPredictEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 17, 49, 10, 755812), 'id_': UUID('1c551765-2ea7-485d-80fc-ca70d7d8baee')}

GetResponseEndEvent
{'timestamp

In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-29db3ffd-8b6d-4fbb-a3b4-8e8083beecc4', parent_id='BaseRetriever.retrieve-a6932e9b-923e-4bdb-bd9b-8d7851487807', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 778394), end_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 963457), duration=0.185063),
 SimpleSpan(id_='BaseRetriever.retrieve-a6932e9b-923e-4bdb-bd9b-8d7851487807', parent_id='RetrieverQueryEngine._query-f115e05c-df1f-4e83-8848-bf0bc5e51f46', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 778061), end_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 964456), duration=0.186395),
 SimpleSpan(id_='LLM.predict-69c3336f-fd4b-4ab2-ac2a-66051620cc7d', parent_id='Refine.get_response-b62868e4-ee86-49b4-85b3-d6a542c9d83c', start_time=datetime.datetime(2024, 3, 11, 17, 49, 5, 980594), end_time=datetime.datetime(2024, 3, 11, 17, 49, 7, 577694), duration=1.5971),
 SimpleSpan(id_='Refine.get_response-b62868e4-ee86-49b4-85b3-d6a542c9d83c', parent_id='CompactAndRefine.get_response-b34a

### Stream Chat

In [ ]:
chat_engine = index.as_chat_engine()

In [ ]:
# response = chat_engine.chat("Tell me a joke.")

In [ ]:
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
streaming_response = chat_engine.stream_chat("Tell me a joke.")

NameError: name 'AgentChatWithStepStartEvents' is not defined

In [ ]:
dispatcher.span_handler.open_spans

{}

In [ ]:
for token in streaming_response.response_gen:
    print(token, end="")

AttributeError: 'NoneType' object has no attribute 'response_gen'

### Very Primitive Tree Viz

In [ ]:
# %pip install treelib -q

In [ ]:
from treelib import Node, Tree

tree = Tree()

In [ ]:
sorted_spans = sorted(
    dispatcher.span_handler.completed_spans, key=lambda x: x.start_time
)

In [ ]:
sorted_spans

[SimpleSpan(id_='AgentRunner.stream_chat-3eeaa89f-0d1b-4e4e-9730-8bd667fe37e4', parent_id=None, start_time=datetime.datetime(2024, 3, 11, 16, 55, 32, 121490), end_time=datetime.datetime(2024, 3, 11, 16, 55, 34, 91977), duration=1.970487),
 SimpleSpan(id_='AgentRunner._chat-8d7be06d-2113-45e4-956a-23d672026ba6', parent_id='AgentRunner.stream_chat-3eeaa89f-0d1b-4e4e-9730-8bd667fe37e4', start_time=datetime.datetime(2024, 3, 11, 16, 55, 32, 121878), end_time=datetime.datetime(2024, 3, 11, 16, 55, 34, 91920), duration=1.970042),
 SimpleSpan(id_='AgentRunner._run_step-a9ee29e6-848e-4606-a0e9-8481ef4b7e4b', parent_id='AgentRunner._chat-8d7be06d-2113-45e4-956a-23d672026ba6', start_time=datetime.datetime(2024, 3, 11, 16, 55, 32, 124157), end_time=datetime.datetime(2024, 3, 11, 16, 55, 33, 715950), duration=1.591793),
 SimpleSpan(id_='OpenAIAgentWorker._run_step-1f1d9b69-ca3d-464b-bfb1-a3a0a9b62186', parent_id='AgentRunner._run_step-a9ee29e6-848e-4606-a0e9-8481ef4b7e4b', start_time=datetime.date

In [ ]:
for span in sorted_spans:
    tree.create_node(
        tag=f"{span.id_} ({span.duration})",
        identifier=span.id_,
        parent=span.parent_id,
        data=span.start_time,
    )

In [ ]:
print(tree.show(stdout=False, sorting=True, key=lambda node: node.data))

AgentRunner.stream_chat-3eeaa89f-0d1b-4e4e-9730-8bd667fe37e4 (1.970487)
└── AgentRunner._chat-8d7be06d-2113-45e4-956a-23d672026ba6 (1.970042)
    ├── AgentRunner._run_step-a9ee29e6-848e-4606-a0e9-8481ef4b7e4b (1.591793)
    │   └── OpenAIAgentWorker._run_step-1f1d9b69-ca3d-464b-bfb1-a3a0a9b62186 (1.590952)
    │       └── OpenAIAgentWorker._call_function-4a83bfe6-f145-4aab-8cd7-9292948a8973 (0.849091)
    │           └── call_function-40673cbe-809b-41f6-8ef7-e1df744b8035 (0.846987)
    │               └── BaseQueryEngine.query-6b840334-9ecc-4bec-a63f-eaeb02264c7f (0.846631)
    │                   └── RetrieverQueryEngine._query-f4e86072-f8f8-4489-bca9-e819b0fcf416 (0.845605)
    │                       ├── BaseRetriever.retrieve-d13492e7-6762-49f7-841f-8b1a926161b8 (0.233974)
    │                       │   └── VectorIndexRetriever._retrieve-457a481a-0b3b-4504-9127-a74da2607971 (0.231577)
    │                       └── BaseSynthesizer.synthesize-bafac62b-d0e4-493e-bac0-884005252cf2 (